### 

In [1]:
df_1 = _dntk.execute_sql(
  'SELECT *\nFROM \'air traffic.csv\'',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_1

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12
3,2003,4,"47,033,260","4,345,444","51,378,704","766,260","55,005","821,265","39,465,980","10,370,592","49,836,572","54,639,679","15,528,761","70,168,440",72.23,66.78,71.02
4,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023,5,"71,423,653","10,358,666","81,782,319","667,331","71,924","739,255","66,743,565","26,805,432","93,548,998","77,821,407","31,950,687","109,772,094",85.77,83.90,85.22
245,2023,6,"72,482,621","11,544,505","84,027,126","661,293","75,279","736,572","68,789,127","29,883,465","98,672,591","78,058,358","33,410,671","111,469,028",88.13,89.44,88.52
246,2023,7,"75,378,157","12,432,615","87,810,772","684,939","79,738","764,677","72,267,904","31,376,000","103,643,904","81,986,010","35,326,191","117,312,202",88.15,88.82,88.35
247,2023,8,"71,477,988","11,572,149","83,050,137","691,482","77,137","768,619","67,933,484","29,938,507","97,871,992","81,997,399","34,908,793","116,906,192",82.85,85.76,83.72


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import datetime
from prophet import Prophet
import calendar
from sklearn.metrics import mean_squared_error, r2_score

### Load CSV

In [3]:
df_air_traffic = pd.read_csv('air traffic.csv')
df_air_traffic.head()

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12
3,2003,4,"47,033,260","4,345,444","51,378,704","766,260","55,005","821,265","39,465,980","10,370,592","49,836,572","54,639,679","15,528,761","70,168,440",72.23,66.78,71.02
4,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07


The CSV file has been saved to the variable `df_air_traffic` in Python.

# Preprocessing

In [4]:
df_air_traffic['Date'] = pd.to_datetime(df_air_traffic[['Year','Month']].assign(DAY=1))
df_air_traffic.head()

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF,Date
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20,2003-01-01
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31,2003-02-01
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12,2003-03-01
3,2003,4,"47,033,260","4,345,444","51,378,704","766,260","55,005","821,265","39,465,980","10,370,592","49,836,572","54,639,679","15,528,761","70,168,440",72.23,66.78,71.02,2003-04-01
4,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07,2003-05-01


Year and month have been combined into one column called "Date"

In [5]:
for col in df_air_traffic.columns:
    if col is object:
        df_air_traffic[col] = df_air_traffic[col].str.replace(',','', regex=False)
        df_air_traffic[col] = pd.to_numeric(df_air_traffic[col])
    print(f"Column: {col}")
    
df_air_traffic.head()

Column: Year
Column: Month
Column: Dom_Pax
Column: Int_Pax
Column: Pax
Column: Dom_Flt
Column: Int_Flt
Column: Flt
Column: Dom_RPM
Column: Int_RPM
Column: RPM
Column: Dom_ASM
Column: Int_ASM
Column: ASM
Column: Dom_LF
Column: Int_LF
Column: LF
Column: Date


,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF,Date
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20,2003-01-01
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31,2003-02-01
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12,2003-03-01
3,2003,4,"47,033,260","4,345,444","51,378,704","766,260","55,005","821,265","39,465,980","10,370,592","49,836,572","54,639,679","15,528,761","70,168,440",72.23,66.78,71.02,2003-04-01
4,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07,2003-05-01


In [6]:
for col in df_air_traffic.columns:
    if df_air_traffic[col].dtype == 'object':
        df_air_traffic[col] = df_air_traffic[col].str.replace(',','')
        df_air_traffic[col] = pd.to_numeric(df_air_traffic[col], errors='coerce')

All commas have been removed from the data

In [7]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

df_air_traffic['Season'] = df_air_traffic['Month'].apply(get_season)
df_air_traffic.head()

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF,Date,Season
0,2003,1,43032450,4905830,47938280,785160,57667,842827,36211422,12885980,49097402,56191300,17968572,74159872,64.44,71.71,66.20,2003-01-01,Winter
1,2003,2,41166780,4245366,45412146,690351,51259,741610,34148439,10715468,44863907,50088434,15587880,65676314,68.18,68.74,68.31,2003-02-01,Winter
2,2003,3,49992700,5008613,55001313,797194,58926,856120,41774564,12567068,54341633,57592901,17753174,75346075,72.53,70.79,72.12,2003-03-01,Spring
3,2003,4,47033260,4345444,51378704,766260,55005,821265,39465980,10370592,49836572,54639679,15528761,70168440,72.23,66.78,71.02,2003-04-01,Spring
4,2003,5,49152352,4610834,53763186,789397,55265,844662,41001934,11575026,52576960,55349897,15629821,70979718,74.08,74.06,74.07,2003-05-01,Spring


Added a column that has categories each row with its corresponding season

In [8]:
_dntk.DeepnoteChart(df_air_traffic, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"axis":{"grid":false,"title":"Year"},"sort":"ascending","type":"temporal","field":"Date","scale":{"type":"linear"},"timeUnit":"year","bandPosition":0},"y":{"axis":{"title":"Number of Flights","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Dom_Flt","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"Season","scale":{"scheme":"category10"}}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Domestic Flights By Year & Season","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Dom_Flt"],"seriesOrder":[0],"specSchemaVersion":2,"tooltipDefaultMode":true}}""")

# Setting Covid and Great Recession

In [9]:
def get_covid(year):
    if year== 2020: 
        return 1
    if year > 2020 or year < 2020: 
        return 0
df_air_traffic["Is_Covid"] = df_air_traffic["Year"].apply(get_covid)

def get_post_covid(year):
    if year <= 2020: 
        return 0
    if year > 2020: 
        return 1
df_air_traffic["Is_Post_Covid"] = df_air_traffic["Year"].apply(get_post_covid)

def get_pre_covid(year):
    if year < 2020: 
        return 1
    if year >= 2020: 
        return 0
df_air_traffic["Is_Pre_Covid"] = df_air_traffic["Year"].apply(get_pre_covid)

def get_gr(month, year):
    if (year >= 2008 and year < 2010):
        if year == 2008:
            return 1
        elif year == 2009 and month <= 6:
            return 1
        else:
            return 0
    else:
        return 0

# Create a lambda function to apply get_gr to both Month and Year columns
df_air_traffic["Is_Recession"] = df_air_traffic.apply(lambda x: get_gr(x["Month"], x["Year"]), axis=1)

df_air_traffic

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,...,ASM,Dom_LF,Int_LF,LF,Date,Season,Is_Covid,Is_Post_Covid,Is_Pre_Covid,Is_Recession
0,2003,1,43032450,4905830,47938280,785160,57667,842827,36211422,12885980,...,74159872,64.44,71.71,66.20,2003-01-01,Winter,0,0,1,0
1,2003,2,41166780,4245366,45412146,690351,51259,741610,34148439,10715468,...,65676314,68.18,68.74,68.31,2003-02-01,Winter,0,0,1,0
2,2003,3,49992700,5008613,55001313,797194,58926,856120,41774564,12567068,...,75346075,72.53,70.79,72.12,2003-03-01,Spring,0,0,1,0
3,2003,4,47033260,4345444,51378704,766260,55005,821265,39465980,10370592,...,70168440,72.23,66.78,71.02,2003-04-01,Spring,0,0,1,0
4,2003,5,49152352,4610834,53763186,789397,55265,844662,41001934,11575026,...,70979718,74.08,74.06,74.07,2003-05-01,Spring,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023,5,71423653,10358666,81782319,667331,71924,739255,66743565,26805432,...,109772094,85.77,83.90,85.22,2023-05-01,Spring,0,1,0,0
245,2023,6,72482621,11544505,84027126,661293,75279,736572,68789127,29883465,...,111469028,88.13,89.44,88.52,2023-06-01,Summer,0,1,0,0
246,2023,7,75378157,12432615,87810772,684939,79738,764677,72267904,31376000,...,117312202,88.15,88.82,88.35,2023-07-01,Summer,0,1,0,0
247,2023,8,71477988,11572149,83050137,691482,77137,768619,67933484,29938507,...,116906192,82.85,85.76,83.72,2023-08-01,Summer,0,1,0,0


One-hot encoded columns to have represent if the row took place during covid or after covid

In [10]:
season_dummies = pd.get_dummies(df_air_traffic["Season"],dtype=int)
df_air_traffic = pd.concat([df_air_traffic, season_dummies], axis=1)

In [11]:
month_dummies = pd.get_dummies(df_air_traffic["Month"], prefix='Month', dtype=int)
df_air_traffic = pd.concat([df_air_traffic, month_dummies], axis=1)
month_dummies.head()

,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0


# Prophet testing

In [12]:
# set start date for prophet forecasting
forecast_start_dt = datetime.date(2022, 1, 1)
forecast_start_date = (forecast_start_dt).strftime("%Y-%m-%d") 
print(forecast_start_date)

2022-01-01


## Setting Training Set and Variables

In [13]:
#Testing Prophet

# rename columns for prophet processing
df_copy = df_air_traffic.copy()
variable = 'Dom_Flt'
df_copy = df_copy.rename(columns={'Date': 'ds', variable: 'y'})
train_set = df_copy

# confirm that the latest date in training set is before the start date
df_sorted = train_set.sort_values(by='ds', ascending=False)
df_sorted

,Year,Month,Dom_Pax,Int_Pax,Pax,y,Int_Flt,Flt,Dom_RPM,Int_RPM,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
248,2023,9,66858490,9392985,76251475,649308,64241,713549,61777546,26076318,...,0,0,0,0,0,0,1,0,0,0
247,2023,8,71477988,11572149,83050137,691482,77137,768619,67933484,29938507,...,0,0,0,0,0,1,0,0,0,0
246,2023,7,75378157,12432615,87810772,684939,79738,764677,72267904,31376000,...,0,0,0,0,1,0,0,0,0,0
245,2023,6,72482621,11544505,84027126,661293,75279,736572,68789127,29883465,...,0,0,0,1,0,0,0,0,0,0
244,2023,5,71423653,10358666,81782319,667331,71924,739255,66743565,26805432,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2003,5,49152352,4610834,53763186,789397,55265,844662,41001934,11575026,...,0,0,1,0,0,0,0,0,0,0
3,2003,4,47033260,4345444,51378704,766260,55005,821265,39465980,10370592,...,0,1,0,0,0,0,0,0,0,0
2,2003,3,49992700,5008613,55001313,797194,58926,856120,41774564,12567068,...,1,0,0,0,0,0,0,0,0,0
1,2003,2,41166780,4245366,45412146,690351,51259,741610,34148439,10715468,...,0,0,0,0,0,0,0,0,0,0


## Run Prophet

In [14]:
# Initialize Prophet model
m = Prophet()

# Adding regressors
m.add_regressor('Is_Pre_Covid', prior_scale= 1.1)
m.add_regressor('Is_Covid', prior_scale=1.0)
m.add_regressor('Is_Post_Covid', prior_scale=0.015)
m.add_regressor('Spring', prior_scale=0.08)
m.add_regressor('Summer', prior_scale= 3.0)
m.add_regressor('Fall', prior_scale=1.0)
m.add_regressor('Winter', prior_scale=0.6)
m.add_regressor('Month_1', prior_scale=0.01)
m.add_regressor('Month_2', prior_scale=0.8)
m.add_regressor('Month_3', prior_scale=1.0)
m.add_regressor('Month_4', prior_scale=0.001)
m.add_regressor('Month_5', prior_scale=0.005)
m.add_regressor('Month_6', prior_scale=0.75)
m.add_regressor('Month_7', prior_scale=0.001)
m.add_regressor('Month_8', prior_scale=0.01)
m.add_regressor('Month_9', prior_scale=0.01)
m.add_regressor('Month_10', prior_scale=0.8)
m.add_regressor('Month_11', prior_scale=0.01)
m.add_regressor('Month_12', prior_scale=0.703125)
m.add_regressor('Is_Recession', prior_scale=0.0001)

# Fit the model with the training data
m.fit(train_set)

prediction_months = 28

# Generate future dataframe with monthly frequency
future = m.make_future_dataframe(periods=prediction_months, freq='MS')  # 'MS' = month start

# Create a date range for the future dates
future_dates = pd.date_range(start=df_air_traffic['Date'].max(), periods=prediction_months + 1, freq='MS')

# Create a DataFrame with future dates and default values for regressors
future_regressors = pd.DataFrame({
    'ds': future_dates,
    'Is_Pre_Covid': 0, # Assuming no pre-COVID period in future
    'Is_Covid': 0,  # Assuming no COVID period in future
    'Is_Post_Covid': 1  # Assuming we're in post-COVID period
})

# Add month and season info
future_regressors['Month'] = pd.to_datetime(future_regressors['ds']).dt.month
future_regressors['Season'] = future_regressors['Month'].apply(get_season)

# Initialize season columns with zeros
future_regressors['Spring'] = 0
future_regressors['Summer'] = 0
future_regressors['Fall'] = 0
future_regressors['Winter'] = 0

future_regressors['Month_1'] = 0
future_regressors['Month_2'] = 0
future_regressors['Month_3'] = 0
future_regressors['Month_4'] = 0
future_regressors['Month_5'] = 0
future_regressors['Month_6'] = 0
future_regressors['Month_7'] = 0
future_regressors['Month_8'] = 0
future_regressors['Month_9'] = 0
future_regressors['Month_10'] = 0
future_regressors['Month_11'] = 0
future_regressors['Month_12'] = 0

future_regressors['Is_Recession'] = 0
future_regressors['Is_Pre_Covid'] = 0
# Set the appropriate season column to 1 based on the Season value
for idx, row in future_regressors.iterrows():
    future_regressors.at[idx, row['Season']] = 1

# Combine the future dates with the regressors
future = future.merge(future_regressors.drop(['Season', 'Month'], axis=1), on='ds', how='left')

# Fill any NaN values that might have been created
future = future.fillna({
    'Is_Pre_Covid': 0,
    'Is_Covid': 0,
    'Is_Post_Covid': 1,
    'Spring': 0,
    'Summer': 0,
    'Fall': 0,
    'Winter': 0,
    'Month_1': 0,
    'Month_2': 0,
    'Month_3': 0,
    'Month_4': 0,
    'Month_5': 0,
    'Month_6': 0,
    'Month_7': 0,
    'Month_8': 0,
    'Month_9': 0,
    'Month_10': 0,
    'Month_11': 0,
    'Month_12': 0,
    'Is_Recession': 0
})

# Predict using the model
fcst_prophet_train = m.predict(future)

# Extract forecasted data that was generated after the forecast date
filter = fcst_prophet_train['ds'] >= forecast_start_date
predicted_df = fcst_prophet_train[filter][['ds', 'yhat']]
predicted_df

18:41:27 - cmdstanpy - INFO - Chain [1] start processing
18:41:28 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
228,2022-01-01,592845.754078
229,2022-02-01,578124.371776
230,2022-03-01,613807.392319
231,2022-04-01,580503.255038
232,2022-05-01,610654.747148
233,2022-06-01,635672.860148
234,2022-07-01,668103.419284
235,2022-08-01,667863.176545
236,2022-09-01,635296.060022
237,2022-10-01,672788.917582


In [15]:
# Merge prediction with actual data to compare accuracy

# Filter data to start date and x months after
end_date = forecast_start_dt + pd.DateOffset(months=prediction_months)

# Filter the dataframe
actual_data = df_air_traffic[
    (df_air_traffic['Date'] >= forecast_start_date) & 
    (df_air_traffic['Date'] < end_date)
]

# Combine
newdf = pd.concat([actual_data['Dom_Flt'], predicted_df], axis=1)
newdf

,Dom_Flt,ds,yhat
228,569018.0,2022-01-01,592845.754078
229,533962.0,2022-02-01,578124.371776
230,629670.0,2022-03-01,613807.392319
231,616135.0,2022-04-01,580503.255038
232,644024.0,2022-05-01,610654.747148
233,640218.0,2022-06-01,635672.860148
234,667889.0,2022-07-01,668103.419284
235,657576.0,2022-08-01,667863.176545
236,622299.0,2022-09-01,635296.060022
237,639282.0,2022-10-01,672788.917582


## Prophet Output

In [16]:
_dntk.DeepnoteChart(newdf, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"line","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":"ascending","type":"temporal","field":"ds","scale":{"type":"linear","zero":false},"timeUnit":"month","bandPosition":0},"y":{"axis":{"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Dom_Flt","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Dom_Flt","scale":{"range":["#2266D3"],"domain":["Dom_Flt"]}}},"transform":[]}]},{"layer":[{"mark":{"clip":true,"type":"line","color":"#83AFF6","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":"ascending","type":"temporal","field":"ds","scale":{"type":"linear","zero":false},"timeUnit":"month","bandPosition":0},"y":{"axis":{"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"yhat","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"yhat","scale":{"range":["#83AFF6"],"domain":["yhat"]}}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Dom_Flt","yhat"],"seriesOrder":[0,1],"specSchemaVersion":2,"tooltipDefaultMode":true}}""")

In [17]:
df_future = pd.read_csv('UPDATE air traffic 2024.csv')
df_future

,Year,Month,DOMESTIC,INTERNATIONAL,TOTAL
0,2024,1,607652,76808,684460
1,2024,2,596253,71827,668080
2,2024,3,675909,81462,757371
3,2024,4,669559,77159,746718
4,2024,5,701690,78281,779971
5,2024,6,706174,82140,788314
6,2024,7,727776,84736,812512
7,2024,8,714725,79201,793926
8,2024,9,675153,64809,739962
9,2024,10,706389,66596,772985


In [18]:
df_2024_dom = df_future['DOMESTIC']
df_2024_dom

0     607652
1     596253
2     675909
3     669559
4     701690
5     706174
6     727776
7     714725
8     675153
9     706389
10    665001
11    681168
Name: DOMESTIC, dtype: int64

In [19]:
df_predicted_2024 = predicted_df[predicted_df['ds'].dt.year == 2024]
df_predicted_2024

,ds,yhat
252,2024-01-01,618035.482206
253,2024-02-01,606748.686140
254,2024-03-01,684058.977114
255,2024-04-01,610645.481435
256,2024-05-01,633078.974802
257,2024-06-01,642266.897895
258,2024-07-01,680959.539847
259,2024-08-01,682096.611339
260,2024-09-01,614793.132135
261,2024-10-01,658400.434262


In [20]:
df_2024_dom = df_2024_dom.reset_index(drop=True)
df_predicted_2024 = df_predicted_2024['yhat'].reset_index(drop=True)

df_compare_2024 = pd.concat([df_2024_dom, df_predicted_2024], axis=1)
df_compare_2024

,DOMESTIC,yhat
0,607652,618035.482206
1,596253,606748.686140
2,675909,684058.977114
3,669559,610645.481435
4,701690,633078.974802
5,706174,642266.897895
6,727776,680959.539847
7,714725,682096.611339
8,675153,614793.132135
9,706389,658400.434262


In [21]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

df_month = pd.DataFrame({'Month': months})

compare_results_df = pd.concat([df_month, df_compare_2024], axis=1)
compare_results_df

,Month,DOMESTIC,yhat
0,Jan,607652,618035.482206
1,Feb,596253,606748.686140
2,Mar,675909,684058.977114
3,Apr,669559,610645.481435
4,May,701690,633078.974802
5,Jun,706174,642266.897895
6,Jul,727776,680959.539847
7,Aug,714725,682096.611339
8,Sep,675153,614793.132135
9,Oct,706389,658400.434262


In [51]:
_dntk.DeepnoteChart(compare_results_df, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"line","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"title":"Month"},"sort":null,"type":"nominal","field":"Month","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Number of Flights","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"DOMESTIC","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Actual","scale":{"range":["#2266D3"],"domain":["Actual"]}}},"transform":[]}]},{"layer":[{"mark":{"clip":true,"type":"line","color":"#83AFF6","tooltip":true},"encoding":{"x":{"axis":{"title":"Month"},"sort":null,"type":"nominal","field":"Month","scale":{"type":"linear","zero":false}},"y":{"axis":{"title":"Number of Flights","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"yhat","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Forecast","scale":{"range":["#83AFF6"],"domain":["Forecast"]}}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"2024 Domestic Flight Forecast Comparison","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Actual","Forecast"],"seriesOrder":[0,1],"specSchemaVersion":2,"tooltipDefaultMode":true}}""")

In [61]:
forecast_mse = mean_squared_error(compare_results_df['DOMESTIC'], compare_results_df['yhat'])
forecast_rmse = np.sqrt(forecast_mse)
forecast_rmsep = np.sqrt(forecast_mse) / compare_results_df['DOMESTIC'].mean() * 100

In [64]:
print(f"Forecast Results Evaluation:\n\nMean Squared Error: {forecast_mse}\nRoot Mean Squared Error: {forecast_rmse}\nRoot Mean Squared Error Percentage: {forecast_rmsep}")

Forecast Results Evaluation:

Mean Squared Error: 2294602544.4540505
Root Mean Squared Error: 47902.00981643725
Root Mean Squared Error Percentage: 7.072626574430021


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b79cc76b-b6e6-4863-afc9-c795d4a1c0ac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>